In [19]:
from datascience import *
from datascience.predicates import are
path_data = '../../../../data/'
import numpy as np
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

from urllib.request import urlopen
import re
from bancor3_simulator.protocol import BancorV3
from bancor3_simulator.core.settings import GlobalSettings
from decimal import Decimal
from fractions import Fraction
import pandas as pd

cooldown_time: int = 7
withdrawal_fee: Decimal = Decimal('0.0025')
whitelisted_tokens: list = ['bnt', 'eth', 'wbtc', 'link']
bnt_funding_limit: Decimal = Decimal('40000')
bnt_min_liquidity: Decimal = Decimal('10000')
price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[1.0],
                            'tkn':[3.0],
                            # 'link':[15.00],
                            # 'eth':[2500.00],
                            # 'wbtc':[40000.00],
                            })

network_fee: Decimal = Decimal('0.2')
trading_fee: Decimal = Decimal('0.01')

# There are other possible configuration settings available, however for the present purpose we will use the defaults.
# v3 = BancorV3(whitelisted_tokens=whitelisted_tokens,
#               network_fee=network_fee,
#               trading_fee=trading_fee,
#               cooldown_time=cooldown_time,
#               withdrawal_fee=withdrawal_fee,
#               bnt_min_liquidity=bnt_min_liquidity,
#               bnt_funding_limit=bnt_funding_limit,
#               price_feeds=price_feeds)

# v3.create_user('Alice')
# v3.create_user('Bob')
# v3.create_user('Charlie')
#
# # The actual deposits.
#
# v3.stake(tkn_amt=100, tkn_name='eth', user_name='Alice')
#
# v3.stake(tkn_amt=10000, tkn_name='link', user_name='Charlie')
#
# v3.stake(tkn_amt=100, tkn_name='wbtc', user_name='Bob')
#
# v3.stake(tkn_amt=1, tkn_name='eth', user_name='Alice')
#
# v3.stake(tkn_amt=1, tkn_name='link', user_name='Charlie')
#
# v3.stake(tkn_amt=1, tkn_name='wbtc', user_name='Bob')
#
# v3.dao_msig_init_pools(whitelisted_tokens)
#
# v3.create_user('Trader')
#
# v3.trade(swap_amount=2000, source_token='bnt',  target_token='link', user_name='Trader', timestep=0)
#
# v3.trade(swap_amount=302.9981, source_token='link',  target_token='bnt', user_name='Trader', timestep=0)
#
# v3.trade(swap_amount=1, source_token='eth',  target_token='wbtc', user_name='Trader', timestep=0)

def read_url(url):
    return re.sub('\\s+', ' ', urlopen(url).read().decode())

In [20]:
# <p align="center">
# <img width="100%" src="https://d9hhrg4mnvzow.cloudfront.net/try.bancor.network/5edb37d6-open-graph-bancor3_1000000000000000000028.png" alt="bancor3" />
# </p>
#
# * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * * *
#
# # Simulation Example

In [21]:
v3 = BancorV3(active_users=['dayTrader', 'tknProvider', 'bntProvider'],
              whitelisted_tokens=['tkn','bnt'])

v3.list_tokens, v3.list_users

(['tkn', 'bnt'], ['dayTrader', 'tknProvider', 'bntProvider'])

In [22]:
v3.upload_json_simulation(path='/UserState/mikewcasale/Local/projects/bancor/simulator-v3/bancor3_simulator/tests/data/scenarios_2.json')
v3.list_tokens, v3.list_users

(['tkn', 'bnt'], ['dayTrader', 'tknProvider', 'bntProvider'])

In [23]:
v3.describe()

,Trading Liquidity,Vault,Staking,ERC20 Contracts,Vortex,External Protection,Protocol WalletState
0,,bnt=0.000000,bnt=0.000000,bnbnt=0.000000,bnt=0.000000,tkn=0.000000,bnbnt=0.000000
1,bnt=0.000000 tkn=0.000000,tkn=0.000000,tkn=0.000000,bntkn=0.000000,,bnt=0.000000,
2,,,,,,,


In [24]:
timestep = 0
simulation = []
validation = []
# QDECIMALS = Decimal(10) ** -18
indx = 0

In [25]:
for transaction in v3.json_data['operations']:
    indx += 1
    transaction_type = transaction['type']
    user_name = transaction['userId']
    mined = transaction['mined']
    amt = transaction['amount'] if 'enable' not in transaction_type else Decimal(0)
    if '{' in str(amt):
        v3.price_feeds = pd.DataFrame({'INDX':[0],
                            'bnt':[Decimal(amt['bntVirtualBalance'])],
                            'tkn':[Decimal(amt['baseTokenVirtualBalance'])],
        })
    else:
        amt = Decimal(amt)
    tkn_name = 'bnt' if 'BNT' in transaction_type else 'tkn'
    target_tkn = 'tkn' if tkn_name == 'bnt' else 'bnt'
    if mined:
        v3.global_state.timestep += 1
        timestep = v3.global_state.timestep
    if 'enable' in transaction_type:
        v3.dao_msig_init_pools(['bnt','tkn'])
    elif 'deposit' in transaction_type:
        v3.stake(tkn_name=tkn_name, tkn_amt=amt, user_name=user_name, timestep=timestep)
    elif 'trade' in transaction_type:
        v3.trade(source_token=tkn_name, target_token=target_tkn, swap_amount=amt, user_name=user_name, timestep=timestep)
    elif 'withdraw' in transaction_type:
        id_number = v3.begin_cooldown(tkn_name=tkn_name,
                                      withdraw_value=amt,
                                      user_name=user_name,
                                      timestep=timestep)
        v3.withdraw(user_name=user_name, id_number=id_number, timestep=timestep)








KeyError: 'bnt'

In [ ]:
# expected = transaction['expected']
# collect = {
#       "indx": [indx],
#       "action": [transaction_type],
#       "amount": [amt],
#       "tkn_dayTrader": [Decimal(expected['tknBalances']['dayTrader']).quantize(QDECIMALS)],
#       "tkn_tknProvider": [Decimal(expected['tknBalances']['tknProvider']).quantize(QDECIMALS)],
#       "tkn_bntProvider": [Decimal(expected['tknBalances']['bntProvider']).quantize(QDECIMALS)],
#       "tkn_masterVault": [Decimal(expected['tknBalances']['masterVault']).quantize(QDECIMALS)],
#       "tkn_epVault": [Decimal(expected['tknBalances']['epVault']).quantize(QDECIMALS)],
#       "bnt_dayTrader": [Decimal(expected['bntBalances']['dayTrader']).quantize(QDECIMALS)],
#       "bnt_tknProvider": [Decimal(expected['bntBalances']['tknProvider']).quantize(QDECIMALS)],
#       "bnt_bntProvider": [Decimal(expected['bntBalances']['bntProvider']).quantize(QDECIMALS)],
#       "bnt_masterVault": [Decimal(expected['bntBalances']['masterVault']).quantize(QDECIMALS)],
#       "bntkn_dayTrader": [Decimal(expected['bntknBalances']['dayTrader']).quantize(QDECIMALS)],
#       "bntkn_tknProvider": [Decimal(expected['bntknBalances']['tknProvider']).quantize(QDECIMALS)],
#       "bntkn_bntProvider": [Decimal(expected['bntknBalances']['bntProvider']).quantize(QDECIMALS)],
#       "bnbnt_dayTrader": [Decimal(expected['bnbntBalances']['dayTrader']).quantize(QDECIMALS)],
#       "bnbnt_tknProvider": [Decimal(expected['bnbntBalances']['tknProvider']).quantize(QDECIMALS)],
#       "bnbnt_bntProvider": [Decimal(expected['bnbntBalances']['bntProvider']).quantize(QDECIMALS)],
#       "bnbnt_bntPool": [Decimal(expected['bnbntBalances']['bntPool']).quantize(QDECIMALS)],
#       "bntCurrentPoolFunding": [Decimal(expected['bntCurrentPoolFunding']).quantize(QDECIMALS)],
#         "tknStakedBalance": [Decimal(expected['tknStakedBalance']).quantize(QDECIMALS)],
#         "bntStakedBalance": [Decimal(expected['bntStakedBalance']).quantize(QDECIMALS)],
#         "tknTradingLiquidity": [Decimal(expected['tknTradingLiquidity']).quantize(QDECIMALS)],
#         "bntTradingLiquidity": [Decimal(expected['bntTradingLiquidity']).quantize(QDECIMALS)],
# }
# validation.append(pd.DataFrame(collect))
#
# collect = {
#       "indx": [indx],
#       "action": [transaction_type],
#       "amount": [amt],
#       "tkn_dayTrader": [v3.global_state.users.users['dayTrader'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
#       "tkn_tknProvider": [v3.global_state.users.users['tknProvider'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
#       "tkn_bntProvider": [v3.global_state.users.users['bntProvider'].wallet['tkn'].tkn_amt.quantize(QDECIMALS)],
#       "tkn_masterVault": [v3.global_state.tokens['tkn'].vault_tkn.quantize(QDECIMALS)],
#       "tkn_epVault": [v3.global_state.tokens['tkn'].external_protection_vault.quantize(QDECIMALS)],
#       "bnt_dayTrader": [v3.global_state.users.users['dayTrader'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
#       "bnt_tknProvider": [v3.global_state.users.users['tknProvider'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
#       "bnt_bntProvider": [v3.global_state.users.users['bntProvider'].wallet['bnt'].tkn_amt.quantize(QDECIMALS)],
#       "bnt_masterVault": [v3.global_state.vault_bnt.quantize(QDECIMALS)],
#       "bntkn_dayTrader": [v3.global_state.users.users['dayTrader'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
#       "bntkn_tknProvider": [v3.global_state.users.users['tknProvider'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
#       "bntkn_bntProvider": [v3.global_state.users.users['bntProvider'].wallet['tkn'].bntkn_amt.quantize(QDECIMALS)],
#       "bnbnt_dayTrader": [v3.global_state.users.users['dayTrader'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
#       "bnbnt_tknProvider": [v3.global_state.users.users['tknProvider'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
#       "bnbnt_bntProvider": [v3.global_state.users.users['bntProvider'].wallet['bnt'].bntkn_amt.quantize(QDECIMALS)],
#       "bnbnt_bntPool": [v3.global_state.pr.quantize(QDECIMALS)],
#         "bntCurrentPoolFunding": [v3.global_state.tokens['tkn'].bnt_funding_amount.quantize(QDECIMALS)],
#         "tknStakedBalance": [v3.global_state.tokens['tkn'].staked_tkn.quantize(QDECIMALS)],
#         "bntStakedBalance": [v3.global_state.staked_bnt.quantize(QDECIMALS)],
#         "tknTradingLiquidity": [v3.global_state.tokens['tkn'].tkn_trading_liquidity.quantize(QDECIMALS)],
#         "bntTradingLiquidity": [v3.global_state.tokens['tkn'].bnt_trading_liquidity.quantize(QDECIMALS)],
# }
# simulation.append(pd.DataFrame(collect))

In [ ]:
# simulation, validation = pd.concat(simulation), pd.concat(validation)
# simulation

In [ ]:
# validation

In [ ]:
# comparison = simulation.compare(validation)
# comparison

In [ ]:
# comparison = comparison.reset_index()
# comparison